In [1]:
%matplotlib inline
import os
import time
import requests
import urllib3
from tqdm import tqdm
from lxml import etree
import pandas as pd
import warnings
import itertools
warnings.filterwarnings('ignore')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [6]:
for year in range(1994, 2018):
    for i in tqdm(range(1, 99)):
        for j in range(1, 99):
            file = "h-tk-%s-%s-%s" %(str(year)+str(year+1), str(i), str(j))
            if not os.path.isfile('HandelingenTK/%s.xml' %file):
                url = "https://zoek.officielebekendmakingen.nl/%s" %file
                resp = requests.get(url + ".xml", verify=False, allow_redirects=False)
                if resp.status_code == 200:
                    hand_file = open('HandelingenTK/%s.xml' %file, 'wb')
                    hand_file.write(resp.content)
                    hand_file.close()
                    resp = requests.get(url+"/metadata.xml", verify=False, \
                                                        allow_redirects=False)
                    meta_file = open('HandelingenTKmeta/%smetadata.xml' %file, 'wb')
                    meta_file.write(resp.content)
                    meta_file.close()    
                else:
                    break

 29%|███████████████████████▍                                                          | 28/98 [21:13<53:04, 45.49s/it]

KeyboardInterrupt: 

In [7]:
enddict = {}
i = 0
for file in tqdm(os.listdir('HandelingenTK/')):
    context = etree.iterparse('HandelingenTK/' + file,  tag='spreekbeurt')
    for _, elem in context:
        surname = elem.findtext('spreker/naam/achternaam')
        party = elem.findtext('spreker/politiek')
        text = ' '.join([elem2.findtext('al') for elem2 \
                         in elem.findall('tekst/al-groep') \
                          if elem2.findtext('al') is not None])
        enddict[i] = [surname, party, text, file]
        i += 1
df = pd.DataFrame.from_dict(enddict, orient='index')
df = df.rename(columns={0:'achternaam', 1:'partij', 2:'tekst', 3:'file'})


100%|███████████████████████████████████████████████████████████████████████████████| 429/429 [00:01<00:00, 291.34it/s]

In [24]:
enddict = {}
for file in tqdm(os.listdir('HandelingenTKmeta')):
    tags = []
    datum = None
    context = etree.iterparse('HandelingenTKmeta/' + file,  tag='metadata')
    for _, elem in context:
        if elem.get('name') == "OVERHEIDop.datumVergadering":
            datum = elem.get('content')
        elif elem.get('name') == "OVERHEIDop.datumVergadering":
            tags.append(elem.get('OVERHEID.category'))
        enddict[file[:-12]+'.xml'] = [datum, tags]
df2 = pd.DataFrame.from_dict(enddict, orient='index')
df2 = df2.rename(columns={0:'datum', 1:'tags'})
len(df2)


100%|██████████████████████████████████████████████████████████████████████████████| 429/429 [00:00<00:00, 1535.17it/s]

429

In [25]:
df.join(df2, on='file').to_csv("Handelingen.csv")